In [51]:
import numpy as np
import polars as pl
from lets_plot import *
LetsPlot.setup_html()

In [52]:
def make_trailing_zeros(arr:np.ndarray)->np.ndarray:
    first_zeros = arr.argmin(axis=1) # first occurences of zeros for each tf (row)
    for i,index in enumerate(first_zeros):
        arr[i,index:] = 0

    return arr

In [53]:
def bounds(arr:np.ndarray,trailing_zero:bool=True)->np.ndarray:
    t0 = arr[:,0] # get the first column t=0
    subbed = np.subtract(arr[:,:].T,t0).T # subtract first column from all columns
    linear_distance = np.abs(subbed) # take the absolute value to get distance
    linear_distance[np.isnan(linear_distance)] = -1 # convert nan to -1 (if remote)
    linear_distance = linear_distance.astype(np.int16) # convert to i16 for tf_indeces are int
    status = linear_distance.copy() # steady (0), close(1) or remote(-1)
    status[(status>1)] = -1 # convert unbounds to -1
    bound = np.zeros_like(status) # from status to bound or not
    bound[(np.isin(status,[0,1]))] = 1 # rest will remain unbound (0)
    if trailing_zero:
        bound = make_trailing_zeros(bound) # once unbound the rest is unbound
    return bound

In [54]:
def remaining(arr:np.ndarray)->np.ndarray:
    arr = bounds(arr)
    print(f"rem1 {arr}")
    counts = np.sum(arr,axis=0) # eaxtract counts of remaining bounds
    print(f"rem2: {counts}")
    return counts

In [55]:
def bounds_dataframe(arr_L:np.ndarray,arr_R:np.ndarray)->pl.DataFrame:
    n_tf,n_timestep = arr_L.shape 
    df = pl.DataFrame()

    # bound status
    bound_all = bounds(arr_L,trailing_zero=False)+bounds(arr_R,trailing_zero=False)
    bound_all[bound_all==2] = 1

    count_L = remaining(arr_L)
    count_R = remaining(arr_R)
    count_all = remaining(bound_all)
    
    df = df.with_columns(
        pl.Series(np.arange(1,n_timestep+1)).alias('timestep'),
        pl.Series(count_L).alias("left"),
        pl.Series(count_R).alias("right"),
        pl.Series(count_all).alias("all"),
    )

    return df

In [56]:
def decay_plot(df:pl.DataFrame,until:int=20):

    df_melted = df.head(until).melt(id_vars="timestep",value_name='counts',variable_name="case")

    p = (
        ggplot(df_melted,aes(x='timestep',y='counts',color='case'))
        + geom_point(size=7)
        + geom_line()
        + scale_color_hue()
    )
    return p

In [57]:
# def main():
positionz = np.load("positions.npz")
L = positionz["L"]
R = positionz["R"]
df = bounds_dataframe(arr_L=L,arr_R=R)
decay_plot(df,until=15)


rem1 [[1 1 1 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]]
rem2: [409 202 112 ...   0   0   0]
rem1 [[1 0 0 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]
rem2: [414 192 114 ...   0   0   0]
rem1 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
rem2: [0 0 0 ... 0 0 0]


C:\Users\zafi_\AppData\Local\Temp\ipykernel_10256\2562807417.py:3: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  df_melted = df.head(until).melt(id_vars="timestep",value_name='counts',variable_name="case")
